In [1]:
import sys
from pathlib import Path

import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier

In [3]:
from src.feature_engineering import get_datasets, partition_data, scale

In [5]:
x_train, y_train, x_test = get_datasets(
    pd.read_csv("data/train.csv"), pd.read_csv("data/test.csv")
    )

/home/eduardo/Documents/simple-vfl/src/feature_engineering.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_combined = df.append(df_test)


In [11]:
n_data = x_train.shape[0]

In [14]:
split_train = partition_data(x_train)
split_test = partition_data(x_test)

In [17]:
split_train

(     Parch  Cabin_A  Cabin_B  Cabin_C  Cabin_D  Cabin_E  Cabin_F  Cabin_G  \
 0        0        0        0        0        0        0        0        0   
 1        0        0        0        1        0        0        0        0   
 2        0        0        0        0        0        0        0        0   
 3        0        0        0        1        0        0        0        0   
 4        0        0        0        0        0        0        0        0   
 ..     ...      ...      ...      ...      ...      ...      ...      ...   
 886      0        0        0        0        0        0        0        0   
 887      0        0        1        0        0        0        0        0   
 888      2        0        0        0        0        0        0        0   
 889      0        0        0        1        0        0        0        0   
 890      0        0        0        0        0        0        0        0   
 
      Cabin_T  Cabin_U  Pclass_1  Pclass_2  Pclass_3  
 0     

In [18]:
outputs = dict()
accuracies = dict()
models = dict()

In [20]:
enumerate(zip(split_train, split_test))

In [19]:
for i, (_train, _test) in enumerate(zip(split_train, split_test)):
    _train, _test = scale(_train, _test)
    _model = LogisticRegression()
    _model.fit(_train, y_train)

    outputs[i] = _model.predict_proba(_train)
    accuracies[i] = 100 * accuracy_score(_model.predict(_train), y_train)
    models[i] = _model

In [29]:
# ----- Combined data stage -----
train_combined = np.empty((n_data, 0))

In [43]:
train_combined

array([[0.7803014 , 0.2196986 , 0.84327655, 0.15672345, 0.76369262,
        0.23630738],
       [0.43107713, 0.56892287, 0.20803364, 0.79196636, 0.37733764,
        0.62266236],
       [0.7803014 , 0.2196986 , 0.2985148 , 0.7014852 , 0.690476  ,
        0.309524  ],
       ...,
       [0.6960638 , 0.3039362 , 0.2985148 , 0.7014852 , 0.8020777 ,
        0.1979223 ],
       [0.43107713, 0.56892287, 0.84327655, 0.15672345, 0.45750704,
        0.54249296],
       [0.7803014 , 0.2196986 , 0.84327655, 0.15672345, 0.53241209,
        0.46758791]])

In [32]:
for _train in outputs.values():
    train_combined = np.hstack((train_combined, _train))

In [50]:
comp_server = MLPClassifier(
    hidden_layer_sizes=(500, 100, 30), learning_rate_init=0.001, verbose=True
)

In [51]:
comp_server.fit(train_combined, y_train)

Iteration 1, loss = 0.64690322
Iteration 2, loss = 0.59384377
Iteration 3, loss = 0.54313898
Iteration 4, loss = 0.49667658
Iteration 5, loss = 0.46146374
Iteration 6, loss = 0.44178425
Iteration 7, loss = 0.43945867
Iteration 8, loss = 0.43755801
Iteration 9, loss = 0.43527510
Iteration 10, loss = 0.43472556
Iteration 11, loss = 0.43464634
Iteration 12, loss = 0.43500888
Iteration 13, loss = 0.43765605
Iteration 14, loss = 0.43193385
Iteration 15, loss = 0.43130967
Iteration 16, loss = 0.43130539
Iteration 17, loss = 0.42976444
Iteration 18, loss = 0.42969765
Iteration 19, loss = 0.42866931
Iteration 20, loss = 0.43013488
Iteration 21, loss = 0.42890203
Iteration 22, loss = 0.42803657
Iteration 23, loss = 0.42943177
Iteration 24, loss = 0.42830433
Iteration 25, loss = 0.42718468
Iteration 26, loss = 0.42662042
Iteration 27, loss = 0.42732365
Iteration 28, loss = 0.42584091
Iteration 29, loss = 0.42745372
Iteration 30, loss = 0.42607035
Iteration 31, loss = 0.42508881
Iteration 32, los

MLPClassifier(hidden_layer_sizes=(500, 100, 30), verbose=True)

In [52]:
pred_train_combined = comp_server.predict(train_combined)
# pred_test_combined = comp_server.predict(test_combined)

In [53]:
train_acc_combined = accuracy_score(pred_train_combined, y_train)

In [54]:
for i, acc in accuracies.items():
    print(f"Holder {i} train accuracy: {acc:.3f}%")

print(f"Combined accuracy: {100*train_acc_combined:.3f}%")

Holder 0 train accuracy: 71.316%
Holder 1 train accuracy: 79.303%
Holder 2 train accuracy: 71.091%
Combined accuracy: 81.552%
